# Skills: Reusable Agent Behaviors

In this notebook, we'll explore how to create, register, and use custom skills to extend agent capabilities with reusable workflows.

## Setup

Configure the environment:

In [ ]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✓ Notebook environment configured")

In [ ]:
import os

# Verify API key
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable.")

## Why Skills?

Skills are reusable agent behaviors that:
- **Encapsulate workflows** - Complex multi-step processes in one call
- **Promote reuse** - Define once, use everywhere
- **Standardize operations** - Consistent behavior across agents
- **Enable composition** - Skills can call other skills
- **Simplify prompts** - Users invoke by name instead of detailed instructions

### Skill vs Custom Tool

| Feature | Skill | Custom Tool |
|---------|-------|-------------|
| Complexity | Multi-step workflows | Single operation |
| Context | Can maintain state | Stateless |
| Agent calls | Can invoke agent | Cannot |
| Use case | "Run test suite" | "Parse JSON" |

## Example 1: Simple Code Quality Skill

Let's create a skill that checks code quality:

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions, Skill

class CodeQualitySkill(Skill):
    """
    A skill that analyzes Python code for quality issues.
    """
    
    @property
    def name(self):
        return "code-quality-check"
    
    @property
    def description(self):
        return "Analyze Python files for code quality issues like long functions, missing docstrings, and complexity"
    
    async def execute(self, file_path: str):
        """
        Execute the code quality check.
        
        Args:
            file_path: Path to the Python file to analyze
        """
        print(f"\n🔍 Running code quality check on: {file_path}\n")
        
        # Use the agent to analyze the file
        prompt = f"""
        Analyze {file_path} for code quality issues:
        
        1. Check for functions longer than 50 lines
        2. Check for missing docstrings
        3. Look for deeply nested code (>3 levels)
        4. Identify duplicate code patterns
        5. Check for overly complex conditionals
        
        Provide a summary with specific line numbers for issues.
        """
        
        result = None
        async for msg in query(
            prompt=prompt,
            options=ClaudeAgentOptions(permission_mode="bypassPermissions")
        ):
            if type(msg).__name__ == "ResultMessage":
                result = msg.result
        
        return result

# Test the skill
skill = CodeQualitySkill()

async def test_code_quality_skill():
    # First find a Python file
    print("Finding a Python file to analyze...\n")
    file_path = None
    
    async for msg in query(
        prompt="Find the first .py file in the current directory tree and give me its path",
        options=ClaudeAgentOptions(permission_mode="bypassPermissions")
    ):
        if type(msg).__name__ == "ResultMessage":
            # Extract file path from result (simplified for demo)
            # In production, you'd parse this more carefully
            result_text = msg.result
            print(f"Found file\n")
    
    # For demo, use a known file
    file_path = "../getting_started.ipynb"  # Analyze the notebook
    
    # Run the skill
    result = await skill.execute(file_path)
    
    print("\n" + "="*60)
    print("QUALITY CHECK RESULTS")
    print("="*60)
    print(result)

await test_code_quality_skill()

### What Makes This a Skill?

1. **Encapsulation**: Complex analysis logic in one callable unit
2. **Reusability**: Can be invoked on any file
3. **Agent integration**: Uses the agent internally for analysis
4. **Clear interface**: Simple `execute(file_path)` method

## Example 2: Test Runner Skill

Create a skill that runs tests and reports results:

In [ ]:
class TestRunnerSkill(Skill):
    """
    A skill that discovers and runs tests in a project.
    """
    
    @property
    def name(self):
        return "test-runner"
    
    @property
    def description(self):
        return "Discover and run all tests in the project, then report results"
    
    async def execute(self, test_pattern: str = "test_*.py"):
        """
        Find and run tests.
        
        Args:
            test_pattern: Glob pattern for test files (default: test_*.py)
        """
        print(f"\n🧪 Running tests matching: {test_pattern}\n")
        
        # Step 1: Find test files
        print("Step 1: Finding test files...")
        test_files = []
        async for msg in query(
            prompt=f"Find all files matching '{test_pattern}' in the current directory tree",
            options=ClaudeAgentOptions(permission_mode="bypassPermissions")
        ):
            if type(msg).__name__ == "ResultMessage":
                print(f"Found files: {msg.result[:200]}...\n")
        
        # Step 2: Run tests (simulated for demo)
        print("Step 2: Running tests...")
        async for msg in query(
            prompt="If there are Python test files, show me what testing framework they use (pytest, unittest, etc.)",
            options=ClaudeAgentOptions(
                continue_conversation=True,
                permission_mode="bypassPermissions"
            )
        ):
            if type(msg).__name__ == "ResultMessage":
                framework = msg.result
        
        # Step 3: Report results
        print("\nStep 3: Test execution complete")
        
        return {
            "status": "completed",
            "framework": framework,
            "pattern": test_pattern
        }

# Test the skill
test_runner = TestRunnerSkill()

async def test_runner_demo():
    result = await test_runner.execute()
    
    print("\n" + "="*60)
    print("TEST RESULTS")
    print("="*60)
    print(f"Status: {result['status']}")
    print(f"Pattern: {result['pattern']}")

await test_runner_demo()

### Multi-Step Skills

This skill demonstrates:
- **Sequential steps**: Find → Run → Report
- **Context persistence**: Uses `continue_conversation`
- **Structured output**: Returns a dictionary with results
- **Flexibility**: Accepts parameters to customize behavior

## Example 3: Documentation Generator Skill

A skill that generates documentation from code:

In [ ]:
class DocGeneratorSkill(Skill):
    """
    Generate documentation for a Python module.
    """
    
    @property
    def name(self):
        return "doc-generator"
    
    @property
    def description(self):
        return "Generate comprehensive documentation for Python modules"
    
    async def execute(self, module_path: str, output_format: str = "markdown"):
        """
        Generate documentation for a module.
        
        Args:
            module_path: Path to the Python module
            output_format: Format for documentation (markdown, rst, html)
        """
        print(f"\n📚 Generating {output_format} documentation for: {module_path}\n")
        
        prompt = f"""
        Generate comprehensive documentation for {module_path}:
        
        1. Read the file and understand its structure
        2. Document all classes with their purpose and methods
        3. Document all functions with parameters and return values
        4. Extract and format docstrings
        5. Create usage examples
        6. Format everything in {output_format}
        
        Provide complete, well-structured documentation.
        """
        
        result = None
        async for msg in query(
            prompt=prompt,
            options=ClaudeAgentOptions(permission_mode="bypassPermissions")
        ):
            if type(msg).__name__ == "ResultMessage":
                result = msg.result
        
        return result

# Test the skill
doc_gen = DocGeneratorSkill()

async def doc_gen_demo():
    # Generate docs for a notebook (simplified)
    result = await doc_gen.execute("session_management.ipynb")
    
    print("\n" + "="*60)
    print("GENERATED DOCUMENTATION")
    print("="*60)
    print(result[:500] + "...")  # Show first 500 chars

await doc_gen_demo()

## Example 4: Registering Skills with the Agent

Skills can be registered and invoked by name:

In [ ]:
from claude_agent_sdk import SkillRegistry

# Create a skill registry
registry = SkillRegistry()

# Register our skills
registry.register(CodeQualitySkill())
registry.register(TestRunnerSkill())
registry.register(DocGeneratorSkill())

print("✓ Registered skills:")
for skill_name in registry.list_skills():
    skill = registry.get_skill(skill_name)
    print(f"  - {skill_name}: {skill.description}")

# Now use skills by name
async def invoke_by_name():
    print("\n" + "="*60)
    print("Invoking skill by name")
    print("="*60)
    
    # Get skill from registry
    skill = registry.get_skill("code-quality-check")
    
    # Execute it
    result = await skill.execute("session_management.ipynb")
    
    print(f"\nResult: {result[:300]}...")

await invoke_by_name()

### Benefits of Registration

- **Discovery**: List available skills
- **Dynamic invocation**: Call by name at runtime
- **Versioning**: Manage multiple versions
- **Sharing**: Export/import skill registries

## Example 5: Composing Skills

Skills can call other skills for complex workflows:

In [ ]:
class FullCodeReviewSkill(Skill):
    """
    Comprehensive code review that uses multiple skills.
    """
    
    def __init__(self, registry: SkillRegistry):
        self.registry = registry
    
    @property
    def name(self):
        return "full-code-review"
    
    @property
    def description(self):
        return "Complete code review including quality check, tests, and documentation"
    
    async def execute(self, file_path: str):
        """
        Perform a full code review.
        """
        print(f"\n📋 Full code review for: {file_path}\n")
        
        results = {}
        
        # Step 1: Quality check
        print("Step 1: Code quality analysis...")
        quality_skill = self.registry.get_skill("code-quality-check")
        results['quality'] = await quality_skill.execute(file_path)
        
        # Step 2: Check tests
        print("\nStep 2: Test coverage...")
        test_skill = self.registry.get_skill("test-runner")
        results['tests'] = await test_skill.execute()
        
        # Step 3: Documentation
        print("\nStep 3: Documentation check...")
        doc_skill = self.registry.get_skill("doc-generator")
        results['docs'] = await doc_skill.execute(file_path)
        
        return results

# Register and test the composite skill
composite_skill = FullCodeReviewSkill(registry)
registry.register(composite_skill)

async def composite_demo():
    result = await composite_skill.execute("session_management.ipynb")
    
    print("\n" + "="*60)
    print("FULL REVIEW COMPLETE")
    print("="*60)
    print(f"Quality: {str(result['quality'])[:100]}...")
    print(f"Tests: {result['tests']}")
    print(f"Docs: {str(result['docs'])[:100]}...")

await composite_demo()

### Skill Composition Patterns

```
Sequential:           Parallel:              Conditional:
Skill A               Skill A                if condition:
  ↓                      ↓                      Skill A
Skill B               Skill B                else:
  ↓                      ↓                      Skill B
Skill C               Skill C
                         ↓
                      Combine
```

## Example 6: Skill with State Management

Skills can maintain state across invocations:

In [ ]:
class ProgressiveRefactorSkill(Skill):
    """
    A skill that refactors code incrementally, tracking progress.
    """
    
    def __init__(self):
        self.state = {
            "files_processed": [],
            "issues_found": 0,
            "issues_fixed": 0,
            "total_runs": 0
        }
    
    @property
    def name(self):
        return "progressive-refactor"
    
    @property
    def description(self):
        return "Incrementally refactor code while tracking progress"
    
    async def execute(self, file_path: str):
        """
        Refactor a file and update state.
        """
        self.state['total_runs'] += 1
        
        print(f"\n🔧 Refactoring: {file_path}")
        print(f"Run #{self.state['total_runs']}\n")
        
        # Analyze file
        async for msg in query(
            prompt=f"Analyze {file_path} and count code quality issues",
            options=ClaudeAgentOptions(permission_mode="bypassPermissions")
        ):
            if type(msg).__name__ == "ResultMessage":
                # Simplified: assume we found some issues
                issues = 3
                self.state['issues_found'] += issues
        
        # Track file
        if file_path not in self.state['files_processed']:
            self.state['files_processed'].append(file_path)
        
        # Simulate fixes
        fixed = 2
        self.state['issues_fixed'] += fixed
        
        print(f"Found {issues} issues, fixed {fixed}\n")
        
        return self.get_progress()
    
    def get_progress(self):
        """Get current refactoring progress."""
        return {
            "files_processed": len(self.state['files_processed']),
            "total_issues": self.state['issues_found'],
            "issues_fixed": self.state['issues_fixed'],
            "issues_remaining": self.state['issues_found'] - self.state['issues_fixed'],
            "completion_rate": f"{(self.state['issues_fixed'] / max(self.state['issues_found'], 1) * 100):.1f}%"
        }
    
    def reset(self):
        """Reset state."""
        self.state = {
            "files_processed": [],
            "issues_found": 0,
            "issues_fixed": 0,
            "total_runs": 0
        }

# Test stateful skill
refactor_skill = ProgressiveRefactorSkill()

async def stateful_demo():
    # Process multiple files
    files = ["session_management.ipynb", "todo_planning.ipynb", "hooks.ipynb"]
    
    for file in files:
        progress = await refactor_skill.execute(file)
        
        print("Current progress:")
        print(f"  Files: {progress['files_processed']}")
        print(f"  Issues: {progress['total_issues']} found, {progress['issues_fixed']} fixed")
        print(f"  Completion: {progress['completion_rate']}")
        print()

await stateful_demo()

### State Management Benefits

- **Progress tracking**: Know where you are in long workflows
- **Resume capability**: Pick up where you left off
- **Metrics collection**: Aggregate data across runs
- **Adaptive behavior**: Change strategy based on past results

## Skill Design Best Practices

### Structure
- **Single responsibility**: One clear purpose per skill
- **Clear naming**: Descriptive, action-oriented names
- **Rich descriptions**: Explain what, when, and why to use
- **Type hints**: Document parameters and return types

### Implementation
- **Error handling**: Always catch and handle exceptions
- **Logging**: Use print or logging for visibility
- **Validation**: Check inputs before processing
- **Idempotency**: Safe to run multiple times

### Composition
- **Loose coupling**: Skills shouldn't depend on specific implementations
- **Registry pattern**: Use for discovery and invocation
- **Dependency injection**: Pass dependencies to constructors
- **Versioning**: Track skill versions for compatibility

### Performance
- **Caching**: Store expensive computations
- **Async all the way**: Use async/await consistently
- **Batch operations**: Process multiple items together
- **Lazy loading**: Don't initialize until needed

## Exercises

Build your own skills:

### Exercise 1: Security Scanner Skill
Create a skill that:
- Scans Python files for security issues
- Checks for hardcoded secrets, SQL injection risks, etc.
- Returns a severity-ranked list of findings
- Suggests fixes for each issue

### Exercise 2: Dependency Analyzer Skill
Build a skill that:
- Finds all import statements in Python files
- Checks which are standard library vs third-party
- Identifies unused imports
- Generates a dependency graph

### Exercise 3: Performance Profiler Skill
Create a skill that:
- Analyzes code for performance issues
- Identifies O(n²) algorithms
- Suggests optimizations
- Tracks improvements across runs (stateful)

### Exercise 4: Composite CI/CD Skill
Build a skill that combines:
- Code quality check (from Example 1)
- Test runner (from Example 2)
- Security scanner (from Exercise 1)
- Returns a pass/fail decision for CI/CD

In [ ]:
# Exercise 1: Your solution here


In [ ]:
# Exercise 2: Your solution here


In [ ]:
# Exercise 3: Your solution here


In [ ]:
# Exercise 4: Your solution here


## Key Takeaways

- **Skills encapsulate reusable agent workflows** in callable units
- **Three core components**: name, description, execute method
- **Skills vs tools**: Skills are multi-step, stateful, can invoke agents
- **Registration enables discovery** and dynamic invocation
- **Composition creates complex workflows** from simple skills
- **State management** allows progressive, resumable operations
- **Best practices**: Single responsibility, error handling, clear interfaces

## Module 2 Complete!

You've now learned:
1. ✅ **Session Management** - Multi-turn conversations with context
2. ✅ **Todo and Planning** - Structured task workflows
3. ✅ **Hooks** - Intercepting and customizing agent behavior
4. ✅ **Skills** - Building reusable agent capabilities

These advanced features let you build sophisticated, production-ready agent applications!